In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import CFEfficacyAgent
from agentic_system.litl_data.litl_utils import load_efficacy_devset

# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Eval Agent")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM("gemini/gemini-2.5-pro", temperature=0.5, cache=False, max_tokens=25000)
)

2025/09/17 15:44:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/17 15:44:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/17 15:44:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [2]:
efficacy_devset = load_efficacy_devset(uniform_efficacy=True)
efficacy_devset = efficacy_devset * 5
len(efficacy_devset)

125

In [3]:
from dspy.evaluate import Evaluate
import random


def efficacy_accuracy(example, pred, trace=None):
    "Calculate error from prediction"
    abs_error = abs(example.cf_efficacy - float(pred.predicted_efficacy))
    accuracy = 1 - abs_error
    return accuracy


N_SAMPLES = 10
efficacy_devset_subset = random.sample(efficacy_devset, N_SAMPLES)
evaluator = Evaluate(
    devset=efficacy_devset_subset,
    num_threads=10,
    display_progress=True,
    provide_traceback=True,
)

efficacy_agent = CFEfficacyAgent()
evaluation = evaluator(efficacy_agent, metric=efficacy_accuracy)

Average Metric: 6.16 / 10 (61.6%): 100%|██████████| 10/10 [01:26<00:00,  8.62s/it]

2025/09/17 15:45:31 INFO dspy.evaluate.evaluate: Average Metric: 6.161973239898488 / 10 (61.6%)


In [4]:
errors = []
confidences = []
for result in evaluation.results:
    errors.append(abs(result[0].cf_efficacy - result[1].predicted_efficacy))
    confidences.append(result[1].confidence)

print("Mean Absolute Error:", sum(errors) / len(errors))
print("Mean Confidence:", sum(confidences) / len(confidences))

Mean Absolute Error: 0.38380267601015117
Mean Confidence: 0.881
